In [128]:
import pandas as pd  
import numpy as np
import seaborn as sns
from plotnine import *
import matplotlib.pyplot as plt
import dateutil.parser
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, roc_curve, accuracy_score, classification_report, confusion_matrix

In [129]:
# data
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [130]:
len('September 29, 2017 ')

19

In [131]:
fake[fake.apply(lambda row: len(row.date) > 18, axis=1)].date

9358     https://100percentfedup.com/served-roy-moore-v...
15507    https://100percentfedup.com/video-hillary-aske...
15508    https://100percentfedup.com/12-yr-old-black-co...
15839    https://fedup.wpengine.com/wp-content/uploads/...
15840    https://fedup.wpengine.com/wp-content/uploads/...
17432    https://fedup.wpengine.com/wp-content/uploads/...
17433    https://fedup.wpengine.com/wp-content/uploads/...
18933    MSNBC HOST Rudely Assumes Steel Worker Would N...
21869    https://fedup.wpengine.com/wp-content/uploads/...
21870    https://fedup.wpengine.com/wp-content/uploads/...
Name: date, dtype: object

In [132]:
true[true.apply(lambda row: len(row.date) > 19, axis=1)].date

Series([], Name: date, dtype: object)

There are multiple date formats like '%d-%b-%y' and '%B %d, %Y' so we should standardise all date entries to a standard format.
<br> But first, we shall remove rows in the 'fake' dataset which do not actually contain dates since the number of these rows is negligible. 

In [133]:
fake = fake[fake.apply(lambda row: len(row.date) < 19, axis=1)]
# 10 rows removed

In [134]:
# converting multiple date formats into a standard format
fake['date'] = fake.apply(lambda row: dateutil.parser.parse(row['date']).strftime("%Y-%m-%d"), axis=1)
fake['date'] = pd.to_datetime(fake.date, format='%Y-%m-%d')

true['date'] = true.apply(lambda row: dateutil.parser.parse(row['date']).strftime("%Y-%m-%d"), axis=1)
true['date'] = pd.to_datetime(true.date, format='%Y-%m-%d')

In [135]:
fake.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23471 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    23471 non-null  object        
 1   text     23471 non-null  object        
 2   subject  23471 non-null  object        
 3   date     23471 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 916.8+ KB


In [136]:
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   title    21417 non-null  object        
 1   text     21417 non-null  object        
 2   subject  21417 non-null  object        
 3   date     21417 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 669.4+ KB


In [137]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,2017-12-31
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,2017-12-31
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,2017-12-30
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,2017-12-29
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,2017-12-25


In [164]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,2017-12-31
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,2017-12-29
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,2017-12-31
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,2017-12-30
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,2017-12-29


In [171]:
fake.groupby('subject').subject.count()

subject
Government News    1568
Middle-east         778
News               9050
US_News             783
left-news          4456
politics           6836
Name: subject, dtype: int64

In [174]:
true.groupby('subject').subject.count()

subject
politicsNews    11272
worldnews       10145
Name: subject, dtype: int64